# TP Final — Phase 1 : Ingestion Bronze (PostgreSQL → MinIO)

**But** : ingerer des donnees brutes depuis PostgreSQL (Northwind) vers la zone **Bronze** (MinIO/S3) selon l’architecture Medallion.

## Attendus couverts
- Lecture generique d’une table PostgreSQL
- Ecriture Parquet dans `s3a://bronze/<table_name>/`
- Partitionnement par **date d’ingestion** au format `YYYY-MM-DD`
- Ajout des metadonnees techniques obligatoires :
  - `_ingestion_timestamp`
  - `_source_system` = `postgresql`
  - `_table_name`

> Remarque : on ajoute aussi `_ingestion_date` pour porter la partition (colonne technique).

## 1) Contexte d’execution
Ce notebook est fait pour etre execute **dans le JupyterLab de l’environnement Docker** (service `jupyter-spark`).

- PostgreSQL dans Docker : `jdbc:postgresql://postgres:5432/app`
- MinIO dans Docker : `http://minio:9000`
- Buckets attendus : `bronze`, `silver`, `gold`

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime

# SparkSession avec config MinIO + drivers (au besoin)
spark = (SparkSession.builder
    .appName("TP Final - Phase 1 Bronze")
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0,org.apache.hadoop:hadoop-aws:3.4.1,com.amazonaws:aws-java-sdk-bundle:1.12.262")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate())

# Pour permettre overwrite uniquement de la partition du jour
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

print("Spark OK")
print("S3A endpoint =", spark.sparkContext._jsc.hadoopConfiguration().get("fs.s3a.endpoint"))

Spark OK
S3A endpoint = http://minio:9000


In [23]:
# Configuration JDBC PostgreSQL
jdbc_url = "jdbc:postgresql://postgres:5432/app"
jdbc_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver",
}

# Date d’ingestion (partition) au format YYYY-MM-DD
ingestion_date = datetime.now().strftime("%Y-%m-%d")
print("Ingestion date =", ingestion_date)

Ingestion date = 2026-01-16


## 2) Fonction d’ingestion generique (PostgreSQL → Bronze)
Cette fonction :
- lit la table via JDBC
- ajoute les colonnes techniques obligatoires
- ecrit en Parquet dans `s3a://bronze/<table_name>/<YYYY-MM-DD>/` (partitionnement par dossier)

In [24]:
def ingest_table_to_bronze(table_name: str, *, ingestion_date: str, base_path: str = "s3a://bronze") -> dict:
    """
    Ingerer une table PostgreSQL vers la zone Bronze sur MinIO.

    Ecrit en Parquet et partitionne par date d’ingestion (YYYY-MM-DD) via la structure de dossiers.
    Ajoute les metadonnees techniques :
      - _ingestion_timestamp
      - _source_system = 'postgresql'
      - _table_name
      - _ingestion_date

    Retourne un petit dictionnaire de stats.
    """
    ingestion_ts_col = F.current_timestamp()

    df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=jdbc_properties)
    row_count = df.count()

    df_out = (df
        .withColumn("_ingestion_timestamp", ingestion_ts_col)
        .withColumn("_source_system", F.lit("postgresql"))
        .withColumn("_table_name", F.lit(table_name))
        .withColumn("_ingestion_date", F.lit(ingestion_date))
    )

    target_path = f"{base_path}/{table_name}/{ingestion_date}"

    df_out.write.mode("overwrite").parquet(target_path)

    return {
        "table": table_name,
        "rows": row_count,
        "path": target_path,
        "partition": ingestion_date,
    }

## 3) Execution Phase 1 : tables obligatoires (+ bonus)
Tables obligatoires : `customers`, `orders`, `order_details`, `products`

Tables bonus (si dispo) : `employees`, `suppliers`, `categories`

In [25]:
required_tables = ["customers", "orders", "order_details", "products"]
bonus_tables = ["employees", "suppliers", "categories"]
tables_to_ingest = required_tables + bonus_tables

results = []
errors = []

print("=== PHASE 1 - INGESTION BRONZE ===")
for table in tables_to_ingest:
    try:
        stats = ingest_table_to_bronze(table, ingestion_date=ingestion_date)
        results.append(stats)
        print(f"[OK] {stats['table']} -> {stats['rows']} lignes -> {stats['path']} (_ingestion_date={stats['partition']})")
    except Exception as e:
        errors.append({"table": table, "error": str(e)})
        print(f"[ERREUR] {table} : {e}")

print("\nResume OK :")
for r in results:
    print(f"  - {r['table']}: {r['rows']} lignes")

if errors:
    print("\nResume erreurs :")
    for err in errors:
        print(f"  - {err['table']}: {err['error']}")

=== PHASE 1 - INGESTION BRONZE ===
[OK] customers -> 91 lignes -> s3a://bronze/customers/2026-01-16 (_ingestion_date=2026-01-16)
[OK] orders -> 830 lignes -> s3a://bronze/orders/2026-01-16 (_ingestion_date=2026-01-16)
[OK] order_details -> 2155 lignes -> s3a://bronze/order_details/2026-01-16 (_ingestion_date=2026-01-16)
[OK] products -> 77 lignes -> s3a://bronze/products/2026-01-16 (_ingestion_date=2026-01-16)
[OK] employees -> 9 lignes -> s3a://bronze/employees/2026-01-16 (_ingestion_date=2026-01-16)
[OK] suppliers -> 29 lignes -> s3a://bronze/suppliers/2026-01-16 (_ingestion_date=2026-01-16)
[OK] categories -> 8 lignes -> s3a://bronze/categories/2026-01-16 (_ingestion_date=2026-01-16)

Resume OK :
  - customers: 91 lignes
  - orders: 830 lignes
  - order_details: 2155 lignes
  - products: 77 lignes
  - employees: 9 lignes
  - suppliers: 29 lignes
  - categories: 8 lignes


## 4) Verifications rapides
On relit une table depuis Bronze et on verifie la presence des colonnes techniques + la partition.

In [26]:
# Exemple : verifier customers (lecture de la partition du jour)
df_bronze_customers = spark.read.parquet(f"s3a://bronze/customers/{ingestion_date}")

expected_cols = {"_ingestion_timestamp", "_source_system", "_table_name", "_ingestion_date"}
missing = expected_cols - set(df_bronze_customers.columns)
print("Colonnes manquantes :", missing)

df_bronze_customers.select(
    "_ingestion_timestamp",
    "_source_system",
    "_table_name",
    "_ingestion_date",
).show(5, truncate=False)

df_bronze_customers.groupBy("_ingestion_date").count().orderBy("_ingestion_date").show(truncate=False)

Colonnes manquantes : set()
+--------------------------+--------------+-----------+---------------+
|_ingestion_timestamp      |_source_system|_table_name|_ingestion_date|
+--------------------------+--------------+-----------+---------------+
|2026-01-16 12:41:52.633452|postgresql    |customers  |2026-01-16     |
|2026-01-16 12:41:52.633452|postgresql    |customers  |2026-01-16     |
|2026-01-16 12:41:52.633452|postgresql    |customers  |2026-01-16     |
|2026-01-16 12:41:52.633452|postgresql    |customers  |2026-01-16     |
|2026-01-16 12:41:52.633452|postgresql    |customers  |2026-01-16     |
+--------------------------+--------------+-----------+---------------+
only showing top 5 rows
+---------------+-----+
|_ingestion_date|count|
+---------------+-----+
|2026-01-16     |91   |
+---------------+-----+



# TP Final — Phase 2 : Raffinement Silver (Bronze → Silver)

Objectif : nettoyer, typer et structurer les donnees Bronze pour les rendre exploitables en Silver.

Sorties attendues (minimum) :
- `dim_customers` : `company_name` en InitCap, `country` en MAJUSCULES
- `dim_products` : jointure produits ↔ categories + `stock_status` (CRITIQUE si stock < 10, sinon NORMAL)
- `fact_orders` : jointure `orders` + `order_details` + calcul `montant_net`

Ecriture en Parquet sous `s3a://silver/<dataset>/<YYYY-MM-DD>/` (meme partition jour que la Phase 1).

> Important : cette Phase 2 lit la partition Bronze du jour `s3a://bronze/<table>/<YYYY-MM-DD>/` produite par la Phase 1.

In [27]:
from pyspark.sql import functions as F

def _first_existing_column(df, candidates: list[str]) -> str:
    for name in candidates:
        if name in df.columns:
            return name
    raise ValueError(f"Aucune colonne trouvee parmi {candidates}. Colonnes dispo: {df.columns}")

def _canonicalize_columns(df, mapping: dict[str, list[str]]):
    # mapping: canonical_name -> [candidates...]
    out = df
    for canonical, candidates in mapping.items():
        found = _first_existing_column(out, candidates)
        if found != canonical:
            out = out.withColumnRenamed(found, canonical)
    return out

def read_bronze(table_name: str, ingestion_date: str):
    path = f"s3a://bronze/{table_name}/{ingestion_date}"
    return spark.read.parquet(path)

def write_silver(df, dataset_name: str, ingestion_date: str, base_path: str = "s3a://silver") -> dict:
    target_path = f"{base_path}/{dataset_name}/{ingestion_date}"
    row_count = df.count()
    df.write.mode("overwrite").parquet(target_path)
    return {"dataset": dataset_name, "rows": row_count, "path": target_path}

In [28]:
# --- Lecture Bronze (partition du jour) ---
customers_bronze = read_bronze("customers", ingestion_date)
orders_bronze = read_bronze("orders", ingestion_date)
order_details_bronze = read_bronze("order_details", ingestion_date)
products_bronze = read_bronze("products", ingestion_date)

# Tables bonus (si presentes) : categories (pour dim_products)
categories_bronze = None
try:
    categories_bronze = read_bronze("categories", ingestion_date)
except Exception as e:
    print("[INFO] categories non disponible en Bronze:", e)

# --- 2.1 Dim_Customers ---
# Règles : company_name InitCap, country UPPER + dedup clé métier
customers = _canonicalize_columns(customers_bronze, {
    "customer_id": ["customer_id", "customerid", "CustomerID"],
    "company_name": ["company_name", "companyname", "CompanyName"],
    "country": ["country", "Country"],
})
dim_customers = (customers
    .dropDuplicates(["customer_id"])
    .withColumn("company_name", F.initcap(F.col("company_name")))
    .withColumn("country", F.upper(F.col("country")))
    .select("customer_id", "company_name", "country")
 )

# --- 2.2 Dim_Products ---
# Règles : join produits ↔ categories + stock_status
products = _canonicalize_columns(products_bronze, {
    "product_id": ["product_id", "productid", "ProductID"],
    "product_name": ["product_name", "productname", "ProductName"],
    "category_id": ["category_id", "categoryid", "CategoryID"],
    "units_in_stock": ["units_in_stock", "unitsinstock", "UnitsInStock"],
})
products = (products
    .withColumn("units_in_stock", F.col("units_in_stock").cast("int"))
    .dropDuplicates(["product_id"])
 )
if categories_bronze is not None:
    categories = _canonicalize_columns(categories_bronze, {
        "category_id": ["category_id", "categoryid", "CategoryID"],
        "category_name": ["category_name", "categoryname", "CategoryName"],
    }).dropDuplicates(["category_id"])
    products = products.join(categories.select("category_id", "category_name"), on="category_id", how="left")
else:
    products = products.withColumn("category_name", F.lit(None).cast("string"))
dim_products = (products
    .withColumn("stock_status", F.when(F.col("units_in_stock") < 10, F.lit("CRITIQUE")).otherwise(F.lit("NORMAL")))
    .select("product_id", "product_name", "category_id", "category_name", "units_in_stock", "stock_status")
 )

# --- 2.3 Fact_Orders ---
# Règles : jointure orders + order_details + montant_net
orders = _canonicalize_columns(orders_bronze, {
    "order_id": ["order_id", "orderid", "OrderID"],
    "customer_id": ["customer_id", "customerid", "CustomerID"],
    "order_date": ["order_date", "orderdate", "OrderDate"],
})
order_details = _canonicalize_columns(order_details_bronze, {
    "order_id": ["order_id", "orderid", "OrderID"],
    "product_id": ["product_id", "productid", "ProductID"],
    "unit_price": ["unit_price", "unitprice", "UnitPrice"],
    "quantity": ["quantity", "Quantity"],
    "discount": ["discount", "Discount"],
})
orders = orders.withColumn("order_date", F.to_date(F.col("order_date")))
order_details = (order_details
    .withColumn("unit_price", F.col("unit_price").cast("double"))
    .withColumn("quantity", F.col("quantity").cast("int"))
    .withColumn("discount", F.col("discount").cast("double"))
    .dropDuplicates(["order_id", "product_id"])
 )
fact_orders = (orders.select("order_id", "customer_id", "order_date")
    .join(order_details, on="order_id", how="inner")
    .withColumn("montant_net", (F.col("unit_price") * F.col("quantity") * (F.lit(1.0) - F.col("discount"))).cast("double"))
    .select("order_id", "product_id", "customer_id", "order_date", "quantity", "unit_price", "discount", "montant_net")
 )

# --- Ecriture Silver ---
results_silver = [
    write_silver(dim_customers, "dim_customers", ingestion_date),
    write_silver(dim_products, "dim_products", ingestion_date),
    write_silver(fact_orders, "fact_orders", ingestion_date),
]
print("=== PHASE 2 - ECRITURE SILVER OK ===")
for r in results_silver:
    print(f"[OK] {r['dataset']} -> {r['rows']} lignes -> {r['path']}")

=== PHASE 2 - ECRITURE SILVER OK ===
[OK] dim_customers -> 91 lignes -> s3a://silver/dim_customers/2026-01-16
[OK] dim_products -> 77 lignes -> s3a://silver/dim_products/2026-01-16
[OK] fact_orders -> 2155 lignes -> s3a://silver/fact_orders/2026-01-16


In [29]:
# Verification rapide : relire une sortie Silver
df_check_dim_customers = spark.read.parquet(f"s3a://silver/dim_customers/{ingestion_date}")
df_check_dim_customers.show(5, truncate=False)
df_check_dim_customers.printSchema()

+-----------+----------------------------------+-------+
|customer_id|company_name                      |country|
+-----------+----------------------------------+-------+
|ALFKI      |Alfreds Futterkiste               |GERMANY|
|ANATR      |Ana Trujillo Emparedados Y Helados|MEXICO |
|ANTON      |Antonio Moreno Taquería           |MEXICO |
|AROUT      |Around The Horn                   |UK     |
|BERGS      |Berglunds Snabbköp                |SWEDEN |
+-----------+----------------------------------+-------+
only showing top 5 rows
root
 |-- customer_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- country: string (nullable = true)

